# Rephrasing

LLMを使用して，低品質な文章を高品質に言い換えるためのNotebookです

Googleドライブをマウント（認証してください）

In [1]:
import sys
from google.colab import drive
from IPython.display import Javascript
drive.mount('/content/drive')

Mounted at /content/drive


パラメータ設定（書き換えてください）

In [4]:
# 取り組むtask_id
task_id = 0

# Notebookを保存したパスを記載
ROOT_PATH = '/content/drive/MyDrive/Colab Notebooks/work/GENIAC/Rephrasing/'

以下2つのセルを実行して放置！

1つのtask_idを完了するのに13時間程度かかります。

途中でセッションが切れた場合は、最初のセルから実行するとcheckpointから再開できます。

Rephrasing前後の文章を確認したい場合は，以下の部分をアンコメントしてください．（出力しながら長時間動かすとブラウザが重くなるかもしれないです）

    print("example['text']=", example['text'])
    print("--------------")
    print("out=", out)
    print("=============")

In [3]:
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'''))

# パッケージのインストール
!pip install gstop loguru transformers sentencepiece accelerate bitsandbytes vllm datasets

<IPython.core.display.Javascript object>

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 96.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [5]:
import os
import json
import random
import re
from pathlib import Path
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from gstop import GenerationStopper

try:
    from vllm import LLM
    from vllm import SamplingParams
    VLLM_AVAILABLE = True
except ImportError:
    VLLM_AVAILABLE = False
    print("VLLM library is not available.")


def setup_model(model_name, enable_vllm):
    if not enable_vllm:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16,
            load_in_4bit=True,
            device_map="auto",
            trust_remote_code=False,
        )
    else:
        tokenizer = AutoTokenizer.from_pretrained(model_name)

        model = LLM(model=model_name)
    return tokenizer, model


def get_data_files(input_dir, extension, task_id):
    input_path = Path(input_dir)
    # ファイル名が '_{task_id}' で終わるファイルを選択
    task_id_str = f"_{task_id}"
    return {file.stem: str(file) for file in input_path.glob(f'*.{extension}') if file.stem.endswith(task_id_str)}


def load_fewshot_examples(file_path, task_type, num_samples=2):
    examples = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # 各行をJSONオブジェクトにデコード
            try:
                example = json.loads(line.strip())
                # タスクタイプに基づいて例をフィルタリング
                if example.get('task') == task_type:
                    examples.append(example)
            except json.JSONDecodeError:
                continue  # 不正なフォーマットの行はスキップ

    # フィルタリングされた例からランダムにサンプルを選択
    selected_examples = random.sample(examples, min(num_samples, len(examples)))  # サンプル数がリスト長を超えないように

    return selected_examples

def create_prompt(fewshot_file: str):
    # 指示を定義
    instructions = [
        {"task": "medium", "text": "次の入力について、ウィキペディアの文章のように、質の高い日本語で多様な言い換えをしてください。できるだけ入力の情報を落とさずに生成してください。"},
        {"task": "qa", "text": "次の入力を、「質問」の後に「回答」が続く2～5個の会話形式に変換してください。できるだけ入力の情報を落とさずに生成してください。"}
    ]

    # # ランダムに指示を選択
    selected_instruction = random.choice(instructions)

    # few-shotの例を読み込む
    # 選択された指示に基づいてfew-shotの例を読み込む
    fewshot_examples = load_fewshot_examples(fewshot_file, selected_instruction['task'], num_samples=2)

    # プロンプトを構築
    prompt = f"""### 指示:
{selected_instruction['text']}

### 入力:
{fewshot_examples[0]["input"]}

### 応答:
{fewshot_examples[0]["output"]}

### 入力:
{fewshot_examples[1]["input"]}

### 応答:
{fewshot_examples[1]["output"]}

"""

    return prompt


def update_text(example: dict, tokenizer, model, fewshot_file: str, enable_gstop: bool, enable_vllm: bool) -> dict:
    # torch.cuda.empty_cache()  # GPUキャッシュのクリア

    # 文字数が2000文字以上の場合は処理をスキップ
    if len(example['text']) >= 2000:
        return example  # 変更なしで返す

    # プロンプトを構築
    prompt = create_prompt(fewshot_file) + f"### 入力:\n{example['text']}\n\n### 応答:\n"
    # print("prompt=", prompt)

    if not enable_vllm:
        # VLLM無効の場合（遅いので今回は使わない）
        input_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
        stopping_criteria = None

        # inputの文字数 * 1.5をmax_new_tokensとする
        max_new_tokens = len(example['text'])*1.5

        # with torch.no_grad():  # 勾配計算を無効化
        if enable_gstop:
            # "###"で生成をストップする場合（gstopライブラリを使用）
            stop_tokens_ids = tokenizer.encode("###", add_special_tokens=False)
            stop_tokens = {"###": [[id] for id in stop_tokens_ids]}
            stopper = GenerationStopper(stop_tokens)
            stopping_criteria = stopper.criteria

        tokens = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,  # 新しく生成するトークンの最大数
            temperature=0.99,
            top_p=0.95,
            do_sample=True,
            stopping_criteria=stopping_criteria
        )

        # プロンプトのトークン数を計算
        prompt_length = len(tokenizer.encode(prompt, add_special_tokens=False))
        # 生成されたトークンからプロンプトに対応する部分を除外してデコード
        out = tokenizer.decode(tokens[0, prompt_length:], skip_special_tokens=True)

    else:
        # VLLMを使用する場合

        # max_tokens（おそらくprompt込み）の指定
        # 2000はおおよそfew-shot文の長さ
        max_tokens = len(example['text'])*2.5 + 2000

        # パラメータ設定
        sampling_params = SamplingParams(temperature=0.8,
                                         top_p=0.95,
                                         max_tokens=max_tokens,
                                         presence_penalty=0.5,
                                         frequency_penalty=0.5,
                                         repetition_penalty=0.8
                                         )

        # "###"でストップする場合
        # gstopは使えないので，stopパラメータをセット
        if enable_gstop:
            stop_strings = "###"
            sampling_params.stop = stop_strings

        outputs = model.generate([prompt], sampling_params=sampling_params)
        out = outputs[0].outputs[0].text

    # 生成されたテキストから文末の改行、"#"、および半角スペースを削除
    out = out.rstrip("\n# ")

    # "質問:" や "回答:" といったプレフィックスと行頭の空白を除去
    out = re.sub(r'^\s*質問:\s*', '', out, flags=re.MULTILINE)
    out = re.sub(r'^\s*回答:\s*', '', out, flags=re.MULTILINE)
    out = re.sub(r'\n\s+', '\n', out)

    # print("example['text']=", example['text'])
    # print("--------------")
    # print("out=", out)
    # print("=============")

    example['text'] = out
    return example

def save_jsonl(data, output_file_path, encoding='utf-8'):

    # 出力ディレクトリが存在しない場合は作成
    output_dir = Path(output_file_path).parent
    os.makedirs(output_dir, exist_ok=True)

    with open(output_file_path, 'w', encoding=encoding) as f:
        for record in data:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')


# バッチファイルを統合し、その後にバッチファイルを削除する
def merge_and_delete_batch_files(output_dir, stem, extension, batches):
    final_output_path = output_dir / f"{stem}_rephrased.{extension}"
    with open(final_output_path, 'w', encoding='utf-8') as final_file:
        for i in range(batches):
            batch_file_path = output_dir / f"{stem}_rephrased_{i}.{extension}"
            with open(batch_file_path, 'r', encoding='utf-8') as batch_file:
                # バッチファイルの内容を最終ファイルに書き込む
                final_file.write(batch_file.read())
            # バッチファイルを削除
            os.remove(batch_file_path)
    # チェックポイントファイルの更新
    checkpoint_file_path = output_dir / f"{stem}_checkpoint.txt"
    with open(checkpoint_file_path, 'w', encoding='utf-8') as f:
        f.write(str(batches - 1)) # 全バッチが処理されたことを示す
    print(f"All batches have been merged and saved to {final_output_path}. Batch files have been deleted.")


def process_and_save_files(model_name, fewshot_file, input_dir, output_dir, enable_gstop=False, enable_vllm=False, batch_size=5):
    tokenizer, model = setup_model(model_name, enable_vllm)

    extension = 'jsonl'
    data_files = get_data_files(input_dir, extension, task_id)  # task_idに応じてファイルを選択
    for stem, file_path in data_files.items():
        dataset = load_dataset("json", data_files=file_path, split='train')

        total_size = len(dataset)
        batches = (total_size - 1) // batch_size + 1  # 総バッチ数を計算

        checkpoint_file = output_dir / f"{stem}_checkpoint.txt"
        if os.path.exists(checkpoint_file):
            with open(checkpoint_file, 'r', encoding='utf-8') as f:
                processed_batches = int(f.read().strip())
            if processed_batches >= batches - 1:
                print(f"All batches for {stem} have been processed. Skipping...")
                continue  # このファイルの全バッチが処理済み

        for i in range(batches):
            if os.path.exists(checkpoint_file):
                with open(checkpoint_file, 'r', encoding='utf-8') as f:
                    processed_batches = int(f.read().strip())
                if i <= processed_batches:
                    continue  # このバッチは既に処理済み

            # torch.cuda.empty_cache()  # GPUキャッシュのクリア

            # 現在のバッチを処理
            batch = dataset.select(range(i * batch_size, min((i + 1) * batch_size, total_size)))
            updated_batch = batch.map(
                update_text,
                fn_kwargs={
                    'tokenizer': tokenizer,
                    'model': model,
                    'fewshot_file': fewshot_file,
                    'enable_gstop': enable_gstop,
                    'enable_vllm': enable_vllm,
                }
            )

            # 更新されたバッチを保存
            output_file_path = output_dir / f"{stem}_rephrased_{i}.{extension}"
            save_jsonl(updated_batch, output_file_path)

            # 処理済みのバッチ数をチェックポイントファイルに書き込み
            with open(checkpoint_file, 'w', encoding='utf-8') as f:
                f.write(str(i))

            print(f"Batch {i+1}/{batches} processed and saved: {output_file_path}")

        # 全バッチの処理後にファイルを統合して削除
        merge_and_delete_batch_files(output_dir, stem, extension, batches)


ROOT_PATH = Path(ROOT_PATH)
INPUT_PATH = ROOT_PATH / 'input'
OUTPUT_PATH = ROOT_PATH / 'output'
FEWSHOT_FILE = ROOT_PATH / 'fewshot_data.json'

if task_id % 2 != 0:
    # task_id が奇数の場合 karasu
    model_name = "lightblue/karasu-7B-chat"
else:
    # task_id が偶数の場合 rakuten
    model_name = "Rakuten/RakutenAI-7B-chat"

enable_gstop = True
enable_vllm = True
batch_size = 5
process_and_save_files(model_name, FEWSHOT_FILE, INPUT_PATH, OUTPUT_PATH, enable_gstop, enable_vllm, batch_size)


tokenizer_config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/755k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.61M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

...

example['text']= options
61枚を もっと見る
オオトヤ ホリデイタウンマツモトテン
大戸屋 ホリデイタウン松本店
rating:4.1
4.1
(口コミ179件)
定食屋・南松本駅から徒歩14分
写真付き口コミを投稿すると最大
1
ホーム
メニュー
写真
写真
口コミ
マップ
写真
すべて
料理
雰囲気
メニュー表
すべて
もっと見る
フィード
発見
検索
マイページ
--------------
out= オオトヤ ホリデイタウンマツモトテン
大戸屋 ホリデイタウン松本店
オオトヤ ホリデイタウンマツモトテン
大戸屋 ホリデイタウン松本店
rating:4.1
4.1
(口コミ179件)
定食屋・南松本駅から徒歩14分
定食屋・南松本駅から徒歩14分
料理の写真をご覧ください。写真付き口コミを投稿すると、最大で3000円分の楽天トラベル旅行割引クーポンが貰えます。
大戸屋 ホリデイタウン松本店で美味しい料理を堪能しました。口コミの写真をご覧ください。料理の写真をご覧ください。フィード発見検索マイページメニュー表すべて料理雰囲気メニュー表すべてもっと見る楽天トラベル旅行割引クーポンが貰えます。南松本駅から徒歩14分です。



Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.66s/it]


example['text']= 根尾あかりのアナル舐め
絶対にナマで連射させてくれる連続中出しソープ 根尾あかり
ECSTASY MASTER 究極のイカセ技マニアックス 根尾あかり
おじさん大好き喜んでいいなりになります 根尾あかり
定年を控えた老いぼれ教師と孤独なおじさんに惹かれる女生徒との性日記。 無邪気な笑みと腰が砕けるほどの濃厚接吻誘惑…教え子に主導権を握られ痴女られる禁断の教淫生活 根尾あかり
本番なしのマットヘルスに行って出てきたのは隣家の高慢な美人妻。弱みを握った僕は本番も中出しも強要！店外でも言いなりの性奴●にした 根尾あかり
彼女の妹に愛されすぎてこっそり子作り性活 根尾あかり
--------------
out= 根尾あかりさんの出演しているAV作品は、どこで見ることができますか？
根尾あかりさんの出演しているAV作品は、AV専門のウェブサイトや、ビデオ・DVDでご覧いただけます。おすすめは、AV専門のウェブサイトです。
根尾あかりさんは、どんな方ですか？
根尾あかりさんは、AV女優さんです。おじさん大好き喜んでいいなりになりますや、絶対にナマで連射させてくれる連続中出しソープなど、多くのAV作品に出演しています。
根尾あかりさんの出演作品では、どのような性活がありますか？
根尾あかりさんの出演作品では、本番なしのマットヘルスに行って出てきたのは隣家の高慢な美人妻。弱みを握った僕は本番も中出しも強要！店外でも言いなりの性奴●にしたや、彼女の妹に愛されすぎてこっそり子作り性活などがあります。



Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

最後までrephrasing処理ができなかった場合は，以下をアンコメントして実行してください．CPUで動作できます！

（タイムオーバーやコンピューティングユニットが尽きた場合など）

In [ ]:
# # 処理済みのテキストと未処理のテキストを結合する
# import os
# import json
# from pathlib import Path
# from datasets import load_dataset

# def get_data_files(input_dir, extension, task_id):
#     input_path = Path(input_dir)
#     # ファイル名が '_{task_id}' で終わるファイルを選択
#     task_id_str = f"_{task_id}"
#     return {file.stem: str(file) for file in input_path.glob(f'*.{extension}') if file.stem.endswith(task_id_str)}

# # バッチファイルを統合し、その後にバッチファイルを削除する
# def merge_and_delete_batch_files(output_dir, stem, extension, batches):
#     final_output_path = output_dir / f"{stem}_rephrased.{extension}"
#     with open(final_output_path, 'w', encoding='utf-8') as final_file:
#         for i in range(batches + 1):
#             batch_file_path = output_dir / f"{stem}_rephrased_{i}.{extension}"
#             if os.path.exists(batch_file_path):
#                 with open(batch_file_path, 'r', encoding='utf-8') as batch_file:
#                     # バッチファイルの内容を最終ファイルに書き込む
#                     final_file.write(batch_file.read())
#                 # バッチファイルを削除
#                 os.remove(batch_file_path)
#     print(f"All batches have been merged and saved to {final_output_path}. Batch files have been deleted.")


# def process_and_save_files(input_dir, output_dir, task_id, batch_size=5):
#     extension = 'jsonl'
#     data_files = get_data_files(input_dir, extension, task_id)
#     for stem, file_path in data_files.items():
#         dataset = load_dataset("json", data_files=file_path, split='train')
#         total_size = len(dataset)
#         batches = (total_size - 1) // batch_size + 1

#         checkpoint_file = output_dir / f"{stem}_checkpoint.txt"
#         processed_batches = 0
#         if os.path.exists(checkpoint_file):
#             with open(checkpoint_file, 'r', encoding='utf-8') as f:
#                 processed_batches = int(f.read().strip())

#         # 処理済みのファイルを統合して保存
#         merge_and_delete_batch_files(output_dir, stem, extension, processed_batches)

#         # 未処理のデータを分割して保存
#         unrephrased_data = dataset.select(range(processed_batches * batch_size, len(dataset)))
#         if unrephrased_data:
#             unrephrased_data.to_json(output_dir / f"{stem}_unrephrased.jsonl")
#             print(f"Saved unrephrased data for {stem}.")

# ROOT_PATH = Path(ROOT_PATH)
# INPUT_PATH = ROOT_PATH / 'input'
# OUTPUT_PATH = ROOT_PATH / 'output_qa_presence_penalty05_frequency_penalty05'
# batch_size = 5

# process_and_save_files(INPUT_PATH, OUTPUT_PATH, task_id)